In [1]:
import pyspark
from pyspark.sql import SQLContext, SparkSession
from pyspark.sql.functions import *

In [2]:
%%sh
echo $JAVA_HOME
java -version

/usr/lib/jvm/java-1.8.0-amazon-corretto


openjdk version "1.8.0_332"
OpenJDK Runtime Environment Corretto-8.332.08.1 (build 1.8.0_332-b08)
OpenJDK 64-Bit Server VM Corretto-8.332.08.1 (build 25.332-b08, mixed mode)


In [3]:
spark_session = SparkSession.builder \
    .config('spark.sql.extension', 'org.apache.spark.sql.hudi.HoodieSparkSessionExtension') \
    .config('spark.jars.packages', 'org.apache.calcite:calcite-core:1.29.0,org.apache.thrift:libfb303:0.9.3,org.datanucleus:datanucleus-core:5.2.9,org.apache.hudi:hudi-utilities-bundle_2.12:0.10.0,org.apache.hudi:hudi-spark3.2.1-bundle_2.12:0.10.1,org.apache.spark:spark-avro_2.12:3.2.1,org.apache.parquet:parquet-hive-bundle:1.11.2,org.apache.hadoop:hadoop-aws:3.2.0,net.java.dev.jets3t:jets3t:0.9.4,com.amazonaws:aws-java-sdk:1.12.200') \
    .config('hive.metastore.client.factory.class', 'com.amazonaws.glue.catalog.metastore.AWSGlueDataCatalogHiveClientFactory') \
    .config('hive.metastore.schema.verification', 'false') \
    .config('spark.delta.logStore.class', 'org.apache.spark.sql.delta.storage.S3SingleDriverLogStore') \
    .config('spark.sql.hive.convertMetastoreParquet', 'false') \
    .config('spark.serializer', 'org.apache.spark.serializer.KryoSerializer') \
    .config('spark.hadoop.fs.s3a.aws.credentials.provider', 'org.apache.hadoop.fs.s3a.SimpleAWSCredentialsProvider') \
    .getOrCreate()

:: loading settings :: url = jar:file:/usr/local/lib/python3.7/site-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
org.apache.calcite#calcite-core added as a dependency
org.apache.thrift#libfb303 added as a dependency
org.datanucleus#datanucleus-core added as a dependency
org.apache.hudi#hudi-utilities-bundle_2.12 added as a dependency
org.apache.hudi#hudi-spark3.2.1-bundle_2.12 added as a dependency
org.apache.spark#spark-avro_2.12 added as a dependency
org.apache.parquet#parquet-hive-bundle added as a dependency
org.apache.hadoop#hadoop-aws added as a dependency
net.java.dev.jets3t#jets3t added as a dependency
com.amazonaws#aws-java-sdk added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-ddb868af-c9b6-43ae-9493-3f8bd36c2e2f;1.0
	confs: [default]
	found org.apache.calcite#calcite-core;1.29.0 in central
	found org.apache.calcite#calcite-linq4j;1.29.0 in central
	found org.apiguardian#apiguardian-api;1.1.0 in central
	found org.checkerframework#checker-qual;3.10.0 in 

RuntimeError: Java gateway process exited before sending its port number

In [3]:
# %%configure -f
# {
#     "conf":  { 
#              "spark.jars":"s3://dl-emr-common-stack-dh-bootstrap-stg/delta-core_2.12-1.0.0.jar",
#              "spark.sql.extensions":"io.delta.sql.DeltaSparkSessionExtension,org.apache.spark.sql.hudi.HoodieSparkSessionExtension",
#              "spark.jars.packages":"io.delta:delta-core_2.12:1.0.0,org.apache.calcite:calcite-core:1.29.0,org.apache.thrift:libfb303:0.9.3,org.datanucleus:datanucleus-core:5.2.9,org.apache.hudi:hudi-utilities-bundle_2.12:0.10.0,org.apache.hudi:hudi-spark3-bundle_2.12:0.10.0,org.apache.spark:spark-avro_2.12:3.1.1,com.amazonaws:aws-java-sdk-bundle:1.12.31",
#              "spark.sql.catalog.spark_catalog":"org.apache.spark.sql.delta.catalog.DeltaCatalog",
#              "hive.metastore.client.factory.class": "com.amazonaws.glue.catalog.metastore.AWSGlueDataCatalogHiveClientFactory",
#              "hive.metastore.schema.verification": "false",
#              "spark.delta.logStore.class": "org.apache.spark.sql.delta.storage.S3SingleDriverLogStore",
#              "spark.serializer":"org.apache.spark.serializer.KryoSerializer",
#              "spark.sql.hive.convertMetastoreParquet":"false",
#         "spark.jars":"hdfs:////apps/hudi/lib/aws-java-sdk-bundle-1.12.31.jar"
#            }
# }

In [ ]:
sc = spark_session.sparkContext
SQLContext = SQLContext(sc)

NameError: name 'spark_session' is not defined

In [ ]:
# Create a DataFrame
inputDF = spark_session.createDataFrame(
    [
         ("Italy", "11", "1111", "table", 5, 1648126827, "2015/05/02", "2015", "05", "02"),
         ("Spain", "22", "2222", "chair", 5, 1648126827, "2015/05/02", "2015", "05", "02")        
    ],
    ["marketplace", "review_id", "customer_id", "product_title", "star_rating", "timestamp", "review_date", "year", "month", "day" ]
)

# Print Schema of inputDF 
inputDF.printSchema()

root
 |-- marketplace: string (nullable = true)
 |-- review_id: string (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- product_title: string (nullable = true)
 |-- star_rating: long (nullable = true)
 |-- timestamp: long (nullable = true)
 |-- review_date: string (nullable = true)
 |-- year: string (nullable = true)
 |-- month: string (nullable = true)
 |-- day: string (nullable = true)



In [ ]:
inputDF.select(col("review_date"),to_date(col("review_date"),"yyyy/MM/dd").alias("review_date"))

DataFrame[review_date: string, review_date: date]

In [7]:
spark_session.sql(
  "create table if not exists default.amazon_customer_review_hudi ( \
      marketplace string, \
      review_id string, \
      customer_id string, \
      product_title string, \
      star_rating int, \
      timestamp long , \
      review_date string, \
      year string, \
      month string, \
      day string \
      ) \
      using hudi \
      location 's3a://pyspark-format-010953350464-ap-southeast-2/amazon_customer_review_hudi/' \
      partitioned by (year,month,day);"
)

22/04/20 06:28:46 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties


Py4JJavaError: An error occurred while calling o44.sql.
: java.lang.NoSuchMethodError: com.amazonaws.http.HttpResponse.getHttpRequest()Lcom/amazonaws/thirdparty/apache/http/client/methods/HttpRequestBase;
	at com.amazonaws.services.s3.internal.S3ObjectResponseHandler.handle(S3ObjectResponseHandler.java:57)
	at com.amazonaws.services.s3.internal.S3ObjectResponseHandler.handle(S3ObjectResponseHandler.java:29)
	at com.amazonaws.http.response.AwsResponseHandlerAdapter.handle(AwsResponseHandlerAdapter.java:69)
	at com.amazonaws.http.AmazonHttpClient$RequestExecutor.handleResponse(AmazonHttpClient.java:1734)
	at com.amazonaws.http.AmazonHttpClient$RequestExecutor.handleSuccessResponse(AmazonHttpClient.java:1454)
	at com.amazonaws.http.AmazonHttpClient$RequestExecutor.executeOneRequest(AmazonHttpClient.java:1369)
	at com.amazonaws.http.AmazonHttpClient$RequestExecutor.executeHelper(AmazonHttpClient.java:1145)
	at com.amazonaws.http.AmazonHttpClient$RequestExecutor.doExecute(AmazonHttpClient.j

In [8]:
# Specify common DataSourceWriteOptions in the single hudiOptions variable
hudiOptions = {
"hoodie.table.name": "amazon_customer_review_hudi",
"hoodie.datasource.write.recordkey.field": "review_id",
"hoodie.datasource.write.partitionpath.field": "year,month,day",
"hoodie.datasource.write.precombine.field": "timestamp",
"hoodie.datasource.write.hive_style_partitioning": "true", 
"hoodie.datasource.write.operation": "upsert", 
"hoodie.datasource.hive_sync.enable": "true",
"hoodie.datasource.hive_sync.table": "amazon_customer_review_hudi",
"hoodie.datasource.hive_sync.partition_fields": "year,month,day",
"hoodie.datasource.hive_sync.partition_extractor_class": "org.apache.hudi.hive.MultiPartKeysValueExtractor",
"hoodie.datasource.hive_sync.use_jdbc":"false",
"hoodie.datasource.hive_sync.mode": "hms"
}


# Create Hudi table and insert data into my_hudi_table_1 hudi table at the S3 location specified 
inputDF.write \
       .format("org.apache.hudi")\
       .option("hoodie.datasource.write.operation", "insert")\
       .options(**hudiOptions)\
       .mode("overwrite")\
       .save("s3a://pyspark-format-010953350464-ap-southeast-2/amazon_customer_review_hudi/")

Py4JJavaError: An error occurred while calling o129.save.
: java.lang.NoSuchMethodError: com.amazonaws.http.HttpResponse.getHttpRequest()Lcom/amazonaws/thirdparty/apache/http/client/methods/HttpRequestBase;
	at com.amazonaws.services.s3.internal.S3ObjectResponseHandler.handle(S3ObjectResponseHandler.java:57)
	at com.amazonaws.services.s3.internal.S3ObjectResponseHandler.handle(S3ObjectResponseHandler.java:29)
	at com.amazonaws.http.response.AwsResponseHandlerAdapter.handle(AwsResponseHandlerAdapter.java:69)
	at com.amazonaws.http.AmazonHttpClient$RequestExecutor.handleResponse(AmazonHttpClient.java:1734)
	at com.amazonaws.http.AmazonHttpClient$RequestExecutor.handleSuccessResponse(AmazonHttpClient.java:1454)
	at com.amazonaws.http.AmazonHttpClient$RequestExecutor.executeOneRequest(AmazonHttpClient.java:1369)
	at com.amazonaws.http.AmazonHttpClient$RequestExecutor.executeHelper(AmazonHttpClient.java:1145)
	at com.amazonaws.http.AmazonHttpClient$RequestExecutor.doExecute(AmazonHttpClient

22/04/20 08:04:40 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 2140626 ms exceeds timeout 120000 ms
22/04/20 08:04:40 WARN SparkContext: Killing executors is not supported by current scheduler.
